# Bibliotecas

In [ ]:
import os, logging, ee, folium, glob, rasterio
import json, datetime, math
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import numpy
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from rasterio.plot import show
from osgeo import ogr, gdal

logging.getLogger('googleapicliet.discovery_cache').setLevel(logging.ERROR)
#GeForce RTX 2070
gpu_dict = {'2070':{'GPU_AFFINTY':1,'GPU_MEMORY_LIMIT_GB':8}}
sel_gpu = '2070'
GPU_AFFINTY  = gpu_dict[sel_gpu]['GPU_AFFINTY'] 
GPU_MEMORY_LIMIT_GB =gpu_dict[sel_gpu]['GPU_MEMORY_LIMIT_GB']

try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.set_visible_devices(gpus[GPU_AFFINTY], 'GPU')
    GPU_MEMORY_LIMIT_GB = GPU_MEMORY_LIMIT_GB * 1e3
    if GPU_MEMORY_LIMIT_GB == 0:
        for gpu in gpus:
          tf.config.experimental.set_memory_growth(gpu,True)
    else:
        tf.config.set_logical_device_configuration(gpus[GPU_AFFINTY],[tf.config.LogicalDeviceConfiguration(memory_limit=GPU_MEMORY_LIMIT_GB)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e: print(e)

# ENV Configs

In [ ]:
GPU_AFFINITY   = gpus[GPU_AFFINTY].name
VERSION        = '2'
MOSAIC_VERSION = '1'
MAPBIOMAS_V    = '8'
GOAL_CLASS     = 'apicum'
BUCKET         = GOAL_CLASS+'_br'
GDRIVE = 'mb'+MAPBIOMAS_V+'-unet-'+ GOAL_CLASS +'-brazil'

GOAL_YEAR      = '2020'

FOLDER_TRAIN   = 'training_samples'
TRAINING_BASE  = 'training_patches_'+ MOSAIC_VERSION
EVAL_BASE      = 'eval_patches_'+ MOSAIC_VERSION

#Local paths
LOCAL_PATH  = '/mnt/storage4/modelos/mb7-unet-'+GOAL_CLASS
MODEL_DIR   = LOCAL_PATH+'/checkpoint/v'+VERSION
OUTPUT_PATH = LOCAL_PATH+'/output/v'+VERSION

# Exportation Configs
BUCKET_patch = BUCKET
FOLDER_patch = 'allPatch'
FOLDER_classification = 'mb'+MAPBIOMAS_V+'_'+GOAL_CLASS+'_'+VERSION

# Specify inputs (Landsat bands)
opticalBands = ['swir1', 'nir', 'red','green','MNDWI','NDVI']
BANDS        = opticalBands
RESPONSE     = 'supervised'
FEATURES     = BANDS + [RESPONSE]

#Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

# Specify model training parameters.
BATCH_SIZE  = 10
DROPOUT     = 0.3
BUFFER_SIZE = 1000
OPTIMIZER   = 'Nadam' 
LOSS        = 'BinaryCrossentropy'
METRICS     = ['RootMeanSquaredError']

# Data Visualization (Supervised Layer)

In [ ]:
baseClassV    = '2'
yearClass_class  = '2020'
yearClass_mosaic = '2020'
version_final = '2'
classID       = 32

supervised_2020 = ee.Image('users/MariaLuizeSolvedCurso/Masters/supervisedImage_unet_mb7_'+GOAL_CLASS+'_'+yearClass_class+'_v' +baseClassV).eq(classID).rename(RESPONSE);
supervisedChannel= supervised_2020.toByte().rename(RESPONSE);

image = ee.Image('users/MariaLuizeSolvedCurso/Masters/mosaic_'+yearClass_mosaic).addBands(supervisedChannel)
mapid=image.getMapId({'bands':['swir1','nir','red'],'min':30,'max':150})
map = folium.Map(location=[-23.0089, -43.6078],zoom_start=13)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Planet', overlay=True,name='Mosaic composite',
  ).add_to(map)
  
mapid=supervisedChannel.select(RESPONSE).getMapId({'min':0,'max':1})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='Apicum '+yearClass_class,
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [ ]:
featureStack = ee.Image.cat([
  image.select(BANDS).unmask(0),
  image.select(RESPONSE).unmask(0)
]).float()

list = ee.List.repeat(1, KERNEL_SIZE)
lists = ee.List.repeat(list, KERNEL_SIZE)
kernel = ee.Kernel.fixed(KERNEL_SIZE, KERNEL_SIZE, lists)
arrays = featureStack.neighborhoodToArray(kernel)

In [ ]:
yearClass_geoms = '2020'
trainingPolys_v1 = ee.FeatureCollection('users/MariaLuizeSolvedCurso/Masters/Geoms/trainPolys_apicum_'+yearClass_geoms+'_v1')
evalPolys_v1     = ee.FeatureCollection('users/MariaLuizeSolvedCurso/Masters/Geoms/testPolys_apicum_'+yearClass_geoms+'_v1')

trainingPolys_v2 = ee.FeatureCollection('users/MariaLuizeSolvedCurso/Masters/Geoms/trainPolys_apicum_'+yearClass_geoms+'_v2_update')
evalPolys_v2     = ee.FeatureCollection('users/MariaLuizeSolvedCurso/Masters/Geoms/testPolys_apicum_'+yearClass_geoms+'_v2_update')

trainingPolys = trainingPolys_v1.merge(trainingPolys_v2)
evalPolys     = evalPolys_v1.merge(evalPolys_v2)
polyImage = ee.Image(0).byte().paint(trainingPolys, 1).paint(evalPolys, 2)
polyImage = polyImage.updateMask(polyImage)

mapid = polyImage.getMapId({'min': 1, 'max': 2, 'palette': ['red', 'blue']})
map = folium.Map(location=[-1.3621, -45.2738], zoom_start=5)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='training polygons',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

# Train/Test Chips Exportation

In [ ]:
version_samples_acc = "2"
# Convert the feature collections to lists for iteration.
trainingPolysList =trainingPolys.toList(trainingPolys.size())
evalPolysList = evalPolys.toList(evalPolys.size())
# These numbers determined experimentally.
n = 20 # Number of shards in each polygon.
N = 200 # Total sample size in each polygon.
FOLDER_TEST    = 'testing_samples'

#Add some generalism
TRAIN_SIZE = trainingPolys.size().getInfo()*N
EVAL_SIZE = evalPolys.size().getInfo()*N
print('TRAIN:'+str(TRAIN_SIZE))
print('EVAL:'+str(EVAL_SIZE))
GDRIVE = 'MB8_Apicum'
# Export all the training data (in many pieces), with one task per geometry.
for g in range(trainingPolys.size().getInfo()):
  geomSample = ee.FeatureCollection([])
  for i in range(n):
    sample = arrays.sample(
      region = ee.Feature(trainingPolysList.get(g)).geometry(), 
      scale = 30, 
      numPixels = N / n, # Size of the shard.
      seed = i,
      tileScale = 8
    )
    geomSample = geomSample.merge(sample)
  
  desc = TRAINING_BASE + '_g' + str(g)
  task = ee.batch.Export.table.toDrive(
    collection = geomSample,
    description = desc, 
    folder = GDRIVE+'/'+FOLDER_TRAIN+'_v'+version_samples_acc, 
    fileNamePrefix = desc,
    fileFormat = 'TFRecord',
    selectors = BANDS + [RESPONSE]
  )
  task.start()

# Export all the evaluation data.
for g in range(evalPolys.size().getInfo()):
  geomSample = ee.FeatureCollection([])
  for i in range(n):
    sample = arrays.sample(
      region = ee.Feature(evalPolysList.get(g)).geometry(), 
      scale = 30, 
      numPixels = N / n,
      seed = i,
      tileScale = 8
    )
    geomSample = geomSample.merge(sample)
  
  desc = EVAL_BASE + '_g' + str(g)
  task = ee.batch.Export.table.toDrive(
    collection = geomSample,
    description = desc, 
    folder = GDRIVE+'/'+FOLDER_TEST+version_samples_acc, 
    fileNamePrefix = desc,
    fileFormat = 'TFRecord',
    selectors = BANDS + [RESPONSE],
  )
  task.start()

In [ ]:
print(trainingPolys.size().getInfo())
print(evalPolys.size().getInfo())

# Datasets Access

In [ ]:
def parse_tfrecord(example_proto):
  """The parsing function.
  Read a serialized example into the structure defined by FEATURES_DICT.
  Args:
    example_proto: a serialized Example.
  Returns: 
    A dictionary of tensors, keyed by feature name.
  """
  print(FEATURES_DICT)
  return tf.io.parse_single_example(example_proto, FEATURES_DICT)



def to_tuple(inputs):
  """Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
  Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
  Args:
    inputs: A dictionary of tensors, keyed by feature name.
  Returns: 
    A dtuple of (inputs, outputs).
  """
  inputsList = [inputs.get(key) for key in FEATURES]
  stacked = tf.stack(inputsList, axis=0)
  # Convert from CHW to HWC
  stacked = tf.transpose(stacked, [1, 2, 0])
  return stacked[:,:,:len(BANDS)], stacked[:,:,len(BANDS):]


def get_dataset(pattern):
  """Function to read, parse and format to tuple a set of input tfrecord files.
  Get all the files matching the pattern, parse and convert to tuple.
  Args:
    pattern: A file pattern to match in a Cloud Storage bucket.
  Returns: 
    A tf.data.Dataset
  """
  # glob = tf.gfile.Glob(pattern) for tendorflow 1.x
  glob = tf.io.gfile.glob(pattern) # for tendorflow 2.x
  dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
  dataset = dataset.map(parse_tfrecord, num_parallel_calls=5)
  dataset = dataset.map(to_tuple, num_parallel_calls=5)
  return dataset

In [ ]:
def get_training_dataset():
    version_samples_acc = "2"
    glob='/home/mluize/modelos/storage/mb7-unet-apicum/train/samples_v'+version_samples_acc+'/'+ TRAINING_BASE + '*'
    dataset = get_dataset(glob)
    dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
    return dataset
training = get_training_dataset()
print(training.take(1))

def get_eval_dataset():
    version_samples_acc = "2"
    glob = '/home/mluize/modelos/storage/mb7-unet-apicum/eval/samples_v'+version_samples_acc+ '/'+ EVAL_BASE + '*'
    dataset = get_dataset(glob)
    dataset = dataset.batch(3).repeat()
    return dataset
evaluation = get_eval_dataset()

# U shaped CNN

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import models
from tensorflow.keras import metrics
from tensorflow.keras import optimizers

def conv_block(input_tensor, num_filters):
    encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
    encoder = layers.BatchNormalization()(encoder)
    encoder = layers.Activation('relu')(encoder)
    encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
    encoder = layers.BatchNormalization()(encoder)
    encoder = layers.Activation('relu')(encoder)
    return encoder

def encoder_block(input_tensor, num_filters):
    encoder = conv_block(input_tensor, num_filters)
    encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
    return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
    decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
    decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
    decoder = layers.BatchNormalization()(decoder)
    decoder = layers.Activation('relu')(decoder)
    decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
    decoder = layers.BatchNormalization()(decoder)
    decoder = layers.Activation('relu')(decoder)
    decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
    decoder = layers.BatchNormalization()(decoder)
    decoder = layers.Activation('relu')(decoder)
    return decoder

def get_model():
    inputs = layers.Input(shape=[None, None, len(BANDS)]) # 256 (shape=[256, 256, len(BANDS)
    encoder0_pool, encoder0 = encoder_block(inputs, 64) # 128
    encoder1_pool, encoder1 = encoder_block(encoder0_pool, 128) # 64
    encoder2_pool, encoder2 = encoder_block(encoder1_pool, 256) # 32
    encoder3_pool, encoder3 = encoder_block(encoder2_pool, 512) # 16
    center = conv_block(encoder3_pool, 1024) # 8 center
    decoder4 = decoder_block(center, encoder3, 512) # 16
    decoder3 = decoder_block(decoder4, encoder2, 256) # 32
    decoder2 = decoder_block(decoder3, encoder1, 128) # 64
    decoder1 = decoder_block(decoder2, encoder0, 64) # 128
    dropout = layers.Dropout(DROPOUT, name="dropout", noise_shape=None, seed=None)(decoder1)
    outputs = layers.Conv2D(1, (1, 1),  activation=tf.nn.sigmoid, padding='same', \
                            kernel_initializer=tf.keras.initializers.GlorotNormal())(dropout) #tensorflow 2.x
    
    model = models.Model(inputs=[inputs], outputs=[outputs])
    optimizer = tf.keras.optimizers.Nadam( 0.000005, name='optimizer')
    
    model.compile(
        optimizer=optimizer, 
        loss=losses.get(LOSS),
        metrics=[metrics.get(metric) for metric in METRICS]
    )
    return model

# Model Selection/Load

In [ ]:
from IPython.display import Image
m = get_model() # UNET
UNET_VERSION = '2'
VERSION = str(VERSION)
EPOCH = 0
CHECK_MODEL_DIR = '/home/mluize/storage4/modelos/mb7-unet-apicum/checkpoint/v'+str(UNET_VERSION)+'/cp-0'+str(EPOCH)+'.ckpt'

In [ ]:
def previewClass(epoch,log):
    counter = 0
    for batch in evaluation.shuffle(1000).take(3):
        pureImage = batch[0]
        supervised = batch[1]
        stacked = tf.transpose(pureImage[0],[0,1,2]).numpy()
        stackedS=tf.transpose(supervised[0],[0,1,2]).numpy()
        test_pred_raw = m.predict(pureImage)
        test_pred_raw = tf.transpose(test_pred_raw[0],[0,1,2]).numpy()
        fig = plt.figure(figsize=[12,4])
        # show original image
        fig.add_subplot(131)
        plt.imshow(stacked[:,:,0:3].astype(np.uint8), \
                   interpolation='nearest', vmin=0, vmax=255)
        fig.add_subplot(132)
        plt.imshow(stackedS[:,:,0], \
                   interpolation='nearest',cmap="gray")
        fig.add_subplot(133)
        plt.imshow(test_pred_raw[:,:,0], \
                   interpolation='nearest',cmap="gray")
        plt.show()
        counter = counter+1 

In [ ]:
lt.style.use("ggplot")
checkpoint_path = CHECK_MODEL_DIR+"/cp-{epoch:04d}.ckpt"
checkpoint_dir  = os.path.dirname(checkpoint_path)

log_dir = '/home/mluize/storage4/modelos/mb7-unet-apicum/output/v'+VERSION

tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir+'/log_model',write_images=True)
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,verbose=1, save_weights_only=True, period=2)
img_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=previewClass)

result = m.fit(x=training,
  epochs=80,
  initial_epoch=0, 
  steps_per_epoch=2000,
  verbose=1,
  shuffle=True,
  validation_data=evaluation,
  validation_steps=10000,
  callbacks = [cp_callback,tensorboard,img_callback])

# Modaic Exportation

In [ ]:
def doExport(out_image_base,index_in, kernel_buffer, roi):
  """Run the image export task.  Block until complete.
  """
  index = index_in
  image = ee.Image('users/MariaLuizeSolvedCurso/Masters/mosaic_'+str(index))
  out_image_base2 = out_image_base+'_'+str(index_in)
  filesList= !ls '/mnt/storage/allPatch/'{out_image_base2}'*'
  print(filesList)
  exportFilesList = [s for s in filesList if out_image_base in s]
  if(len(exportFilesList) > 1):
    print('Image Already Exported')
    return None
  print("Exporting..") 
  print(BANDS)
  task = ee.batch.Export.image.toDrive(
    image = image.select(BANDS).toFloat(), 
    description = out_image_base+'_'+str(index), 
    folder = 'mosaics_unet_'+str(index), 
    fileNamePrefix = out_image_base+'_'+str(index), 
    region = roi, 
    scale = 30, 
    fileFormat = 'TFRecord', 
    maxPixels = 1e13,
    formatOptions = { 
      'patchDimensions': KERNEL_SHAPE,
      'kernelSize': kernel_buffer,
      'compressed': True,
      'maxFileSize': 104857600
    }
  )
  task.start()

  print('Running image export to Cloud Storage...')
  import time
  time.sleep(1)
  # Error condition
  if task.status()['state'] != 'COMPLETED':
    print('Error with image export.')
  else:
    print('Image export completed.')

# Mosaic Patches Prediction

In [ ]:
def doPrediction(out_image_base,index_in,region_id, user_folder, kernel_buffer, region):
  """Perform inference on exported imagery, upload to Earth Engine.
  """
  out_image_base = out_image_base+'_'+str(index_in)
  filesList = glob.glob("/home/mluize/mosaics_landsat/"+str(index_in)+"/"+out_image_base+"*")

  print(out_image_base)
  rasterFolder = OUTPUT_PATH  + '/classifications_tiff/'+str(index_in)
  rasterURILZW = rasterFolder + '/outimage_'+VERSION+'_'+str(region_id)+'_'+str(index_in)+'_lzw.tif'

  if os.path.exists(rasterURILZW):
        print('Arquivo ja predito \n\n')
        return None
    
  if(len(filesList) == 0):
    print('Sem arquivos')
    !echo 'ERRO! (Sem arquivos) grid={y} - {region_id}' >> log.log
    return None
  exportFilesList = [s for s in filesList if out_image_base in s]    
    

  # Get the list of image files and the JSON mixer file.
  imageFilesList = []
  jsonFile = None
  for f in exportFilesList:
    if f.endswith('.tfrecord.gz'):
      imageFilesList.append(f)
    elif f.endswith('.json'):
      jsonFile = f

  # Make sure the files are in the right order.
  imageFilesList.sort()
  
  predictioned_file =  !gsutil ls 'gs://'{BUCKET_patch}'/'{FOLDER_classification}'/'{VERSION}'/'{out_image_base}'.TFRecord'
  out_image_asset = user_folder + '/' + out_image_base
  out_image_file = 'gs://' + BUCKET_patch + '/' + FOLDER_classification + '/'+VERSION+'/' + out_image_base + '.TFRecord'
  
  # Load the contents of the mixer file to a JSON object.
  jsonText = open(jsonFile)
  # Get a single string w/ newlines from the IPython.utils.text.SList
  mixer = json.load(jsonText)
  
  patches = mixer['totalPatches']
  cols = int(mixer["patchesPerRow"])
  rows = int(mixer["totalPatches"]/cols)
  
  # Get set up for prediction.
  x_buffer = int(kernel_buffer[0] / 2)
  y_buffer = int(kernel_buffer[1] / 2)
  
  buffered_shape = [
      KERNEL_SHAPE[0] + kernel_buffer[0],
      KERNEL_SHAPE[1] + kernel_buffer[1]]

  imageColumns = [
    tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32) #Tensorflow 2.x
      for k in BANDS
  ]  
    
  imageFeaturesDict = dict(zip(BANDS, imageColumns))
  def parse_image(example_proto):
    return tf.io.parse_single_example(example_proto, imageFeaturesDict) #Tensorflow 2.x

  def toTupleImage(inputs):
    inputsList = [inputs[key] for key in BANDS] #BANDS
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked
  
  # Create a dataset from the TFRecord file(s) in Cloud Storage.
  imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')
  imageDataset = imageDataset.map(parse_image, num_parallel_calls=4)
  imageDataset = imageDataset.map(toTupleImage).batch(1)
    
  # Perform inference.
  predictions = m.predict(imageDataset, steps=patches, verbose=2)
  patchesPerRow  = mixer['patchesPerRow']
  TotalPatches   = mixer['totalPatches']
  patchDimension = mixer['patchDimensions']

  #Manipulating Prediction Numpy Array OUTPUT
  counter       = 1
  rowCounter    = 1
  globalCounter = 0
  finalArray    = numpy.array([])

  rowArray = numpy.array([])
  for raw_record in predictions:
      raw_record = numpy.squeeze(raw_record) #Exclude Bands
      rows,cols = raw_record.shape
      raw_record = raw_record[128:384,128:384]
      if rowCounter == 1:
          finalArray = rowArray
      if counter <= patchesPerRow:
          if counter == 1:
              rowArray = raw_record
          else:
              rowArray = numpy.concatenate((rowArray,raw_record), axis = 1)
          counter = counter+1
      else:
          counter = 2
          rowCounter = rowCounter+1
          if numpy.array_equal(finalArray,rowArray):
              finalArray = rowArray
          else:
              finalArray = numpy.concatenate((finalArray,rowArray),axis=0)
          rowArray = raw_record
      globalCounter = globalCounter+1
  finalArray = numpy.concatenate((finalArray,rowArray),axis=0)
  show(Image.fromarray(finalArray))
  rows,cols = finalArray.shape

  driver = gdal.GetDriverByName("GTiff")

  finalArray2  = numpy.array([finalArray])
  rasterFolder = OUTPUT_PATH  + '/classifications_tiff/'+str(index_in)

  if not os.path.exists(rasterFolder):
    print('lets make the directory')
    os.makedirs(rasterFolder)
  
  rasterURI    = rasterFolder + '/UNET_v'+VERSION+'_grid_'+str(region_id)+'_year_'+str(index_in)+'.tif'
  rasterURILZW = rasterFolder + '/outimage_'+VERSION+'_'+str(region_id)+'_'+str(index_in)+'_lzw.tif'
 
  with rasterio.open(rasterURI,'w',
          driver="GTiff",
          height=rows,
          width=cols,
          count=1,
          dtype="float32",
          crs=mixer["projection"]["crs"],
          transform=mixer["projection"]["affine"]["doubleMatrix"],
          nodata="nan") as dataset:
              dataset.write(finalArray2)
  !gdal_translate -of GTiff -co "COMPRESS=LZW" -co "PREDICTOR=2" -co "TILED=YES" {rasterURI} {rasterURILZW}
  !rm {rasterURI}
  out_image_asset = user_folder + '/' + out_image_base
    
  print('Writing predictions...')
  out_image_file  = rasterFolder+'/' + out_image_base + '.TFRecord'
  out_image_mixer = rasterFolder+'/'+ out_image_base + '.json'
  #------------TFRECORD WRITE ------------
  writer = tf.io.TFRecordWriter(out_image_file)
  patches = 0
  for predictionPatch in predictions:
    predictionPatch = predictionPatch[
        x_buffer:x_buffer+KERNEL_SIZE, y_buffer:y_buffer+KERNEL_SIZE]

    # Create an example.
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'classification': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=predictionPatch.flatten()))
        }
      )
    )
    # Write the example.
    writer.write(example.SerializeToString())
    patches += 1

  writer.close()
  !cp {jsonFile} {out_image_mixer}


#   UPLOAD TFRecord TO BUCKET, THEN TO GEE
  !gsutil -o GSUtil:parallel_composite_upload_threshold=150M cp  {out_image_file} {gcs_path}
  !gsutil cp {jsonFile} {gcs_json}
  print('gsutil cp '+jsonFile+' '+gcs_json)
  
  # Start the upload.
  print('earthengine upload image  --asset_id='+out_image_asset+''+gcs_path+' '+gcs_json+')\n')  
  print('out_image_asset: '+out_image_asset+'\n')
  print('gcs_path: '+gcs_path+'\n')
  print('gcs_json: '+gcs_json+'\n')
  !earthengine upload image   --asset_id={out_image_asset} {gcs_path} {gcs_json}

# Grid

In [ ]:
import pygeoj
kernel_buffer = [256, 256]
image_base_name = 'allPatch_UNET_grid_'
grid = pygeoj.load('/home/mluize/storage4/modelos/GRID/GRID-ALLCALSSES-COL8-19052023.geojson')
print("Total Features on GRID")
print(len(grid))

# Mosaic Exportation Process

In [ ]:
# Run the export.
for region in grid:
    region_id = int(region.properties['id'])
    if int(region_id) > 0 and region.properties['apicum'] == 1 and (region_id == 1975):
      print('Region:')
      print(region.geometry.coordinates)
      for y in range(2020, 2021):
          doExport(image_base_name+str(region_id)+str('_' + MOSAIC_VERSION),y, kernel_buffer, region.geometry.coordinates[0])

# Prediction Process

In [ ]:
import time
image_base_name = 'allPatch_UNET_grid_'
MOSAIC_VERSION  = '1'
print(MOSAIC_VERSION)
for y in  range(2020, 2021):
    start = time.time()
    print('starting...')
    !echo 'year={y}' >> log.log
    processed_grids = []
    for region in grid:
        region_id = region.properties['id']
        region_id = int(region_id)
        if region.properties['apicum'] == 1 and not (region_id in processed_grids):
            processed_grids.append(region_id)
            print(f"region: {region_id}, year: {y}")
            user_folder = 'projects/solvedltda/assets/MB7_Apicum/v2/unet_prediction'
            print('Predicting')
            doPrediction(image_base_name+str(region_id)+str('_' + MOSAIC_VERSION),y,region_id, user_folder, kernel_buffer, region.geometry.coordinates)
            print('Finish')
            !echo 'grid={y} -{region_id}' >> log.log
    end = time.time()
    print('Prediction Time per year = '+str(end - start))
print('DONE')

In [ ]:
!gsutil -o GSUtil:parallel_composite_upload_threshold=15M -m cp -n /home/mluize/storage4/modelos/mb7-unet-apicum/output/v2/classifications_tiff/2020/*.tif gs://mineracao_mb8/classification_mb8_apicum/
!ls -1 /home/mluize/storage4/modelos/mb7-unet-apicum/output/v2/classifications_tiff/2020/*.tif > lista
!sh geeUpload.sh